In [99]:
from bs4 import BeautifulSoup
import re
from urllib import request
from pprint import pprint
from datetime import datetime as dt
import time
import pandas as pd

In [28]:
start_url = 'http://www.cambsfire.gov.uk/news-and-incidents/incidents.aspx'
req = request.Request(start_url, headers={'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.67 Safari/537.36'})
content = request.urlopen(req)
p1_soup = BeautifulSoup(content, 'html.parser')

### Define functions

In [60]:
def listpage_scrape(list_page_url):
    '''function takes the url of a page at 
    http://www.cambsfire.gov.uk/news-and-incidents/
    and returns a list of dictionaries with 
    the incident title, date and text'''
    ua_header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.67 Safari/537.36'}
    
    req = request.Request(list_page_url, headers=ua_header)
    content = request.urlopen(req)
    list_page_soup = BeautifulSoup(content, 'html.parser').find('div', class_='main-content')

    incids_list = []

    try:
        for event in list_page_soup.findAll('div', class_='row')[:-1]: # rows are events, last row is navigation pane
            url = 'http://www.cambsfire.gov.uk' + event.find('a')['href']
            req = request.Request(url, headers=ua_header)
            content = request.urlopen(req)
            soup = BeautifulSoup(content, 'html.parser')
        
            incid_dict = {}
        
            incid_dict['title'] = soup.find('h1', class_='content-title').text
            incid_dict['date'] = soup.find('p', class_='content-date').text
            incid_dict['text'] = ' '.join([p.text for p in soup.find('div', class_='main-content-body').findAll('p')]).replace(u'\xa0', u' ')
            
            incids_list.append(incid_dict)
            
            #time.sleep(0.5)    
    except:
        pass
    
    return incids_list

In [81]:
def get_page_urls(url):
    '''function takes the url of a page at 
    http://www.cambsfire.gov.uk/news-and-incidents/
    and returns a list of the urls of the pages 
    linked to in the navigation bar at the bottom'''
    
    ua_header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.67 Safari/537.36'}
    
    req = request.Request(url, headers=ua_header)
    content = request.urlopen(req)
    soup = BeautifulSoup(content, 'html.parser')
    
    url_list = []
    for page_url in soup.find('div', class_='cfrs-pagenumbers').findAll('a')[:-1]:
        url_list.append('http://www.cambsfire.gov.uk' + page_url['href'])
    return url_list

In [83]:
def get_archive_urls(url):
    '''function takes the url of a page at 
    http://www.cambsfire.gov.uk/news-and-incidents/
    and returns a list of the urls of the pages 
    linked to in the navigation bar at the bottom'''
    ua_header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.67 Safari/537.36'}
    
    req = request.Request(url, headers=ua_header)
    content = request.urlopen(req)
    soup = BeautifulSoup(content, 'html.parser')
    
    archive_urls = soup.findAll('div', class_='cfrs-area-navigation')[1].findAll('a')
    archive_urls = ['http://www.cambsfire.gov.uk' + a['href'] for a in archive_urls]
    return archive_urls

### Scrape the data

In [78]:
# this is the page with the most recent incident reports on
start_page_url = 'http://www.cambsfire.gov.uk/news-and-incidents/incidents.aspx'

ua_header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.67 Safari/537.36'}
    
req = request.Request(testpageurl, headers=ua_header)
content = request.urlopen(req)
start_page_soup = BeautifulSoup(content, 'html.parser')


In [80]:
# The archive urls are scraped from the initial incident report page
archive_urls = start_page_soup.findAll('div', class_='cfrs-area-navigation')[1].findAll('a')
archive_urls = ['http://www.cambsfire.gov.uk' + a['href'] for a in archive_urls]


In [92]:
start_page_url = 'http://www.cambsfire.gov.uk/news-and-incidents/incidents.aspx'

allrecords = []

# this is a list of the root pages (the start page of most recent incidents and all of the archives)
root_page_urls = [start_page_url] +  get_archive_urls(start_page_url)

# this iterates through all of the root page urls, visiting each page listed in the navigation pane at the bottom
# and scrapes information from each of the record links contained in these list pages
for root_page in root_page_urls:
    for records_page in get_page_urls(root_page):
        allrecords = allrecords + listpage_scrape(records_page)


In [104]:
pd.DataFrame(allrecords).to_csv(r'C:\git\fire\data\Fire_Regions\CambridgeshireFire.csv')